This activity was one the assessment for the Machine Learning Subject in torrens university.

The purpose of this notebook is to make prediction on the bike sharing dataset.

Here are used different models and techniques such as:
Random forest,
Decision trees,
Gradient boosting,
Xgboost,
Hyperparameter tuning with GridSearchCV, 
Cross validation.

**Import libreries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

**Loading the datasets**: For MLN 601 a repository in Github was created to serve the purpose of storing the datasets used in the assessments. 
There are two datasets related to Bike sharing. One includes the 24 hours of the day for each day and the other doesn't include this column.

In [ ]:
url_1= 'https://raw.githubusercontent.com/anfelipe31/csv-files/master/hour.csv'
hour = pd.read_csv(url_1,sep=",")

url_2 = 'https://raw.githubusercontent.com/anfelipe31/csv-files/master/day.csv'
day = pd.read_csv(url_2,sep=",")

In [ ]:
day

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,1/01/2011,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2/01/2011,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,3/01/2011,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,4/01/2011,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,5/01/2011,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,27/12/2012,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,28/12/2012,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,29/12/2012,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,30/12/2012,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796


In [ ]:
hour

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,1/01/2011,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,1/01/2011,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,1/01/2011,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,1/01/2011,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,1/01/2011,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,31/12/2012,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,31/12/2012,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,31/12/2012,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,31/12/2012,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


In [ ]:
day.shape


(731, 16)

In [ ]:
day.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [ ]:
hour.shape

(17379, 17)

In [ ]:
hour.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [ ]:
day.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [ ]:
hour.describe()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.0000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,8690.0000,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,5017.0295,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.0000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4345.5000,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,8690.0000,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,13034.5000,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000
max,17379.0000,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000


In [ ]:
day.isnull().sum()

In [ ]:
day.info()

In [ ]:
hour.isnull().sum()

In [ ]:
hour.info()

# Dropping columns that do not have relevance in the analysis

In [ ]:
day.drop(['instant','dteday','yr','casual','registered'], axis=1, inplace=True)
day

# Normalize column cnt dividing by 1000, replace cnt by ncnt: normalized count

In [ ]:
day['ncnt']=day['cnt'].div(1000)
day.drop(['cnt'], axis=1, inplace=True)
day

Distributions

In [ ]:
day.hist(bins=100, density=True,figsize=(20,15))
plt.show()

correlation coefficients

In [ ]:
corr_matrix=day.corr()
corr_matrix['ncnt'].sort_values(ascending=False)

In [ ]:
sns.pairplot(day, diag_kind='kde', kind='reg', plot_kws=dict(marker="+")) #, linewidth=1))

In [ ]:
corr=day.corr()
plt.figure(figsize=(15,15))
ax=sns.heatmap(corr,cmap=sns.diverging_palette(155,265,s=100,l=40,n=12),annot=True,center=0)

# Dropping the columns with correlation factor smaller than 10%

In [ ]:
day.drop(['holiday','weekday','workingday'], axis=1, inplace=True)
day

# Splitting the dataset in traning and testing subsets

In [ ]:
x=day[['season','mnth','weathersit','temp','atemp','hum','windspeed']]
y=day['ncnt']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)

# This function allows me to get the values when I apply cross validation

In [ ]:
def display_scores(scores):
  print("Scores:", scores)
  print("mean:", scores.mean())
  print("{Standars deviation:", scores.std())

# DECISION TREE

In [ ]:
tree_reg = DecisionTreeRegressor("friedman_mse",min_weight_fraction_leaf=0.11)
tree_reg.fit(x_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='friedman_mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.11, presort='deprecated',
                      random_state=None, splitter='best')

# Applying cross validation in the training set

In [ ]:
scores=cross_val_score(tree_reg, x_train,y_train,scoring="neg_mean_squared_error",cv=10)
tree_rms_scores = np.sqrt(-scores)


# Calling the fuction display_scores to get the values from the cross validation

In [ ]:
display_scores(tree_rms_scores)

Scores: [1.34425709 1.41368204 1.46304477 1.2727475  1.42591289 1.35120932
 1.33264451 1.32567347 1.44505387 1.38993311]
mean: 1.376415854923081
{Standars deviation: 0.05757133925154594


# RANDOM FOREST

In [ ]:
random_reg=RandomForestRegressor(criterion="mse")
random_reg.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

# Applying cross validation in the training set

In [ ]:
scores=cross_val_score(random_reg, x_train,y_train,scoring="neg_mean_squared_error",cv=10)
random_reg_scores = np.sqrt(-scores)


# Calling the function display_scores to get the values from the cross validation

In [ ]:
display_scores(random_reg_scores)

Scores: [1.1336014  1.20592224 1.36564205 1.07472095 1.24176313 1.22728799
 1.13136885 1.22314427 1.15333616 1.26532052]
mean: 1.202210756700423
{Standars deviation: 0.07855321166788945


# Gradient Boosting

In [ ]:
gbrt=GradientBoostingRegressor(n_estimators=125, learning_rate=0.1,criterion="friedman_mse", min_weight_fraction_leaf=0.1)
gbrt.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.1, n_estimators=125,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

# Applying cross validation in the training set

In [ ]:
scores=cross_val_score(gbrt, x_train,y_train,scoring="neg_mean_squared_error",cv=10)
gbrt_scores = np.sqrt(-scores)


# Calling the function display_scores to get the values from the cross validation

In [ ]:
display_scores(gbrt_scores)

Scores: [1.13901186 1.29444234 1.42662524 1.08876672 1.25621228 1.29416698
 1.18613872 1.22271541 1.2522094  1.2468469 ]
mean: 1.2407135856231464
{Standars deviation: 0.08800602809077411


# XGBOOST

In [ ]:
xgb_reg=xgb.XGBRegressor(objective="reg:linear")
xgb_reg.fit(x_train,y_train)

[03:33:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

# Applying cross validation in the training set

In [ ]:
scores=cross_val_score(xgb_reg, x_train,y_train,scoring="neg_mean_squared_error",cv=10)
xgb_reg_scores = np.sqrt(-scores)


[03:33:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:33:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:33:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:33:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:33:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:33:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:33:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:33:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

# Calling the function display_scores to get the values from the cross validation

In [ ]:
display_scores(xgb_reg_scores)

Scores: [1.17072829 1.2245446  1.37160967 1.1057323  1.29652108 1.2665291
 1.12874197 1.25492062 1.27378748 1.23246967]
mean: 1.232558478344074
{Standars deviation: 0.07584872923475665


# Tuning the models
# Taken from: Machine learning tutorial python 16: Hyperparameter tuning(GridsearchCV) - youtube.com
# Here, potential parameters are defined for each model.

It takes so long to compute these two block of code

In [ ]:
model_params = {
    "tree_reg":{
        "model":DecisionTreeRegressor(),
        "params":{"criterion":["friedman_mse","mse","mae"],
                  "min_weight_fraction_leaf":[0.01,0.02,0.03]
            
        }
    },
    "random_reg":{
        "model":RandomForestRegressor(),
        "params":{"criterion":["mse","mae"],
                  "min_weight_fraction_leaf":[0.03,0.04,0.05],
                  "n_estimators":[115,116,118,119]
        }
        
    },
    "gbrt":{
        "model":GradientBoostingRegressor(),
        "params":{"criterion":["friedman_mse","mse","mae"],
                  "min_weight_fraction_leaf":[0.04,0.05,0.06],
                  "n_estimators":[88,89,90,91]
        }
    },
    "xgb_reg":{
        "model":xgb.XGBRegressor(objective='reg:linear'),
        "params":{"n_stimators":[40,43,48,57,65]
        }
    }
}

# Here, each model is avaluated with every possible combination of paramenters. then, best combination of parameters is returned for each model

In [ ]:
scores=[]
for model_name, mp in model_params.items():
  reg=GridSearchCV(mp['model'], mp['params'],cv=10,scoring="neg_mean_squared_error",return_train_score=False)
  reg.fit(x_train,y_train)
  scores.append({
      "model":model_name,
      "best_score":reg.best_score_,
      "best_params":reg.best_params_
  })



[03:40:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:40:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:40:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:40:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:40:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:40:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:40:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:40:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

# Here, the results obtained are converted in a dataframe to facilitate the reading

In [ ]:
scores_df=pd.DataFrame(scores, columns=["model","best_score","best_params"])
scores_df["max_score"]=scores_df["best_score"].div(-1)
scores_df.drop(["best_score"], axis=1, inplace=True)
scores_df["best_score"]=scores_df["max_score"].pow(1/2)
scores_df.drop(["max_score"], axis=1, inplace=True)
scores_df

,model,best_params,best_score
0,tree_reg,"{'criterion': 'friedman_mse', 'min_weight_frac...",1.314833
1,random_reg,"{'criterion': 'mse', 'min_weight_fraction_leaf...",1.214687
2,gbrt,"{'criterion': 'mse', 'min_weight_fraction_leaf...",1.229342
3,xgb_reg,{'n_stimators': 40},1.234890


# As it is not possible to read the second column of the scores dataframe, I export this dataframe to google drive. Now I can read it from a spreadsheet

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
scores_df.to_csv('data.csv')
!cp data.csv "drive/My Drive/"

# Now, the values obtained from the step befored are introduced again in the models. following the same procedure.
# 1) Training the model
# 2) Applying croos validation in the training set.
# 3) Calling the function display_scores
# 4) Calculate the score on the testing set.

# Decision Tree

In [ ]:
tree_reg = DecisionTreeRegressor("friedman_mse",min_weight_fraction_leaf=0.04)
tree_reg.fit(x_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='friedman_mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.04, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
scores=cross_val_score(tree_reg, x_train,y_train,scoring="neg_mean_squared_error",cv=10)
tree_rms_scores = np.sqrt(-scores)


In [ ]:
display_scores(tree_rms_scores)

Scores: [1.17302543 1.33143349 1.38703049 1.1922297  1.40104583 1.25796162
 1.1306899  1.31454095 1.46301624 1.35773435]
mean: 1.3008707988868438
{Standars deviation: 0.10346697476400468


In [ ]:
y_predict_tree=tree_reg.predict(x_test)
tree_reg.score(x_test,y_test)

0.46913647152848437

# Random Forest

In [ ]:
random_reg=RandomForestRegressor(criterion="mse", min_weight_fraction_leaf=0.03,n_estimators=116)
random_reg.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.03,
                      n_estimators=116, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
scores=cross_val_score(random_reg, x_train,y_train,scoring="neg_mean_squared_error",cv=10)
random_reg_scores = np.sqrt(-scores)

In [ ]:
display_scores(random_reg_scores)

Scores: [1.13419471 1.24407963 1.3397807  1.08815015 1.28586226 1.23556072
 1.12059231 1.24058965 1.22358764 1.22634349]
mean: 1.213874125693532
{Standars deviation: 0.07364632819695284


In [ ]:
y_predict_forest=random_reg.predict(x_test)
random_reg.score(x_test,y_test)

0.5705516180474361

# Gradient Boost

In [ ]:
gbrt=GradientBoostingRegressor(n_estimators=90, learning_rate=0.1,criterion="mse", min_weight_fraction_leaf=0.09)
gbrt.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.09, n_estimators=90,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
scores=cross_val_score(gbrt, x_train,y_train,scoring="neg_mean_squared_error",cv=10)
gbrt_scores = np.sqrt(-scores)

In [ ]:
display_scores(gbrt_scores)

Scores: [1.13138269 1.2797447  1.39601957 1.10792736 1.25419905 1.27549632
 1.17292022 1.24564715 1.21704267 1.24015105]
mean: 1.2320530766820563
{Standars deviation: 0.07813537344925318


In [ ]:
y_predict_gbrt=gbrt.predict(x_test)
gbrt.score(x_test,y_test)

0.606578036010993

# XGBOOST

In [ ]:
xgb_reg=xgb.XGBRegressor(n_estimators=48,objective="reg:linear" )
xgb_reg.fit(x_train,y_train)

[03:43:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=48,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
scores=cross_val_score(xgb_reg, x_train,y_train,scoring="neg_mean_squared_error",cv=10)
xgb_reg_scores = np.sqrt(-scores)

[03:43:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:43:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:43:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:43:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:43:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:43:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:43:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:43:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
display_scores(xgb_reg_scores)

Scores: [1.16926126 1.2213251  1.31335484 1.10681357 1.30654011 1.25781874
 1.13462813 1.25373818 1.24270071 1.21429742]
mean: 1.2220478046480288
{Standars deviation: 0.06477224998626142


In [ ]:
y_predict_xgb=xgb_reg.predict(x_test)
xgb_reg.score(x_test,y_test)

0.6040391798470119

# convert the predictions for each model in a dataframe

In [ ]:
y_predict_tree_df=pd.DataFrame(y_predict_tree)
y_predict_tree_df

In [ ]:
y_predict_forest_df=pd.DataFrame(y_predict_forest)
y_predict_forest_df

In [ ]:
y_predict_gbrt_df=pd.DataFrame(y_predict_gbrt)
y_predict_gbrt_df

In [ ]:
y_predict_xgb_df=pd.DataFrame(y_predict_xgb)
y_predict_xgb_df

# create a dataframe with the predictions of each model.
# the order of the columns as follow: Decision Tree - Random Forest - Gradient Boosting - XGBoost

Having a da taframe for the predicted results allows to compare results.

In [ ]:
predictions = pd.concat([y_predict_tree_df,y_predict_forest_df,y_predict_gbrt_df,y_predict_xgb_df],  names=["Decision Tree","Random Forest","Gradient Boost", "XGBOOST"],axis=1)
predictions[20:50]

,0,0,0,0
20,5.864250,6.040483,5.913658,6.017069
21,5.203242,5.446425,5.985639,4.538937
22,3.790366,4.165791,4.160151,4.188660
23,1.403000,1.924365,1.945771,2.035258
24,2.621394,2.288745,1.920031,1.749058
25,5.542364,5.710059,5.718528,5.792010
26,2.451091,2.378279,2.451480,2.767000
27,5.430483,6.241984,6.774257,6.436366
28,2.621394,2.386570,2.420944,2.472190
29,4.740421,4.278703,4.478793,4.143142
